# Generative Advarsarial Network 

Here is presented the first GAN created by Ian Goodfellow https://arxiv.org/abs/1406.2661

Implementation is done in tensorflow

Created by: Tiago Almeida 02/02/2018

In [1]:
##imports
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
import util as ut # auxiliar file to help in data visualization

#tensorflow version when notebook was created - 1.4.0
tf.__version__


'1.4.0'

### Data MNIST

In [14]:
mnist_flat_size = 784

#mnist data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data")

print(mnist.train.images[0])

#reset graph using during notebook development
tf.reset_default_graph()

Extracting ./data\train-images-idx3-ubyte.gz
Extracting ./data\train-labels-idx1-ubyte.gz
Extracting ./data\t10k-images-idx3-ubyte.gz
Extracting ./data\t10k-labels-idx1-ubyte.gz
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.

### Generator network 
In this implementation i use tensorflow layers API for creating the network

The network receive a 100-dimensinal vector (z) and return a 728-vector (same size of mnist image (28x28)). The objective is the generator create a mapping between space of vector z and the real MNIST DATA! (i think :P)

In [3]:
#array that contain the network dimention at least 2 dimention (input and output dimentions)
G_dimentions = [100,128,mnist_flat_size]

Z = tf.placeholder(tf.float32, shape=[None, G_dimentions[0]], name='x_generator_input')

def generator(x):
    with tf.name_scope("generator_nn"):
        g_net = tf.layers.dense(x,G_dimentions[0],
                                    activation=tf.nn.relu,
                                    name='g_input', reuse=tf.AUTO_REUSE)
        for i in range(1,len(G_dimentions)-1):#loop throw hidden layers
            g_net = tf.layers.dense(g_net,G_dimentions[i],
                                    activation=tf.nn.relu,
                                    name='g_hidden'+str(i),reuse=tf.AUTO_REUSE)
        g_net = tf.layers.dense(g_net,G_dimentions[-1],activation=tf.nn.sigmoid,name='g_output',reuse=tf.AUTO_REUSE)
        return g_net


### Discriminator network 
In this implementation i use tensorflow layers API for creating the network

The network receives a image (728 vector) and try classify as fake or original.

In [4]:
#array that contain the network dimention at least 2 dimention (input and output dimentions)
D_dimentions = [mnist_flat_size,128,1] ##ouput 1-fake, 0-true

X = tf.placeholder(tf.float32, shape=[None, D_dimentions[0]], name='x_discriminator_input')

def discriminator(x):
    with tf.name_scope("discriminator_nn"):
        d_net = tf.layers.dense(x,units=D_dimentions[0],
                                    activation=tf.nn.relu,
                                    name='d_input',reuse=tf.AUTO_REUSE)
        for i in range(1,len(D_dimentions)-1):#loop throw hidden layers
            d_net = tf.layers.dense(d_net,D_dimentions[i],
                                    activation=tf.nn.relu,
                                    name='d_hidden'+str(i),reuse=tf.AUTO_REUSE)
        d_net = tf.layers.dense(d_net,D_dimentions[-1],activation=None,name='d_output',reuse=tf.AUTO_REUSE)
        #save the logits before the sigmoid activation (ouput between[0,1])
        d_logits = d_net
        d_net = tf.nn.sigmoid(d_net)
        return d_net

### Loss functions Generator and Discriminator

Optimizers of tensorflow dont have maximizing method so we minimize the negative loss.

In [6]:

with tf.name_scope("generator_loss"): #D(G(Z))
    d_fake_prob = discriminator(generator(Z))
    g_loss = tf.reduce_mean(tf.log(d_fake_prob))

with tf.name_scope("discriminator_loss"):
    d_true_prob = discriminator(X)
    d_loss = tf.reduce_mean(tf.log(d_true_prob) + tf.log(1. - d_fake_prob))


#### Geting the trainable variables for generator and the discriminator (disadvantage of using layers API)

Since we dont want to obtimize the discriminator when training de generator and vice-versa

In [7]:
#
generator_variables = [var.name for var in tf.trainable_variables() if 'g_' in var.name]
discriminator_variables = [var.name for var in tf.trainable_variables() if 'd_' in var.name]

# kernel = weight
print(generator_variables)
print(discriminator_variables)



['g_input/kernel:0', 'g_input/bias:0', 'g_hidden1/kernel:0', 'g_hidden1/bias:0', 'g_output/kernel:0', 'g_output/bias:0']
['d_input/kernel:0', 'd_input/bias:0', 'd_hidden1/kernel:0', 'd_hidden1/bias:0', 'd_output/kernel:0', 'd_output/bias:0']


### Training Algorithm


In [ ]:
lr=0.01
with tf.name_scope("discriminator_train"):
    d_optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    d_train_op = d_optimizer.minimize(-d_loss, var_list=discriminator_variables) # note minimizing negative loss is the same as maximizing, tf dont have maximizing

with tf.name_scope("generator_train"):
    g_optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    g_train_op = g_optimizer.minimize(-g_loss,var_list=generator_variables)


## Start graph computations and algorithm
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    
    
    